# FACE ID App

## Imports 

In [1]:
import cv2

import os.path
from os import path
import numpy as np


import warnings

from keras_vggface import VGGFace
from keras_vggface import utils

import tensorflow

from keras.preprocessing import image
from sklearn.metrics import r2_score

## Step 1 : Learning your face 

Load a few pictures of yourself (5 for example), *alone on the picture*, with your face visible. Name them : image_1.jpg , image_2.jpg etc.

Load them in a new folder called "Your_name" in the directory "Raw_images" 

## Face detection 

In [2]:
# Give your name in input, with the first letter in capital letter
your_name = 'Adrien'

#Number of raw images of yourself you have submitted
nb_photos = 6

In [4]:
os.mkdir('Face_extract\\'+your_name)

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

#Warning
warnings.warn("Close the images windows each time one opens ")


for j in range(nb_photos):

# Read the input image

    img = cv2.imread(rf'.\Raw_images\\'+ your_name +'\image_'+str(j+1)+'.jpg')
    cv2.imshow('Visualisation',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 10)

# Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
#Save the face
    for i in range(len(faces)) :
        (x, y, w, h) = faces[i]  
        cv2.imwrite(rf'Face_extract\\'+your_name +'\\face_' +str(j+1)+ '.png', img[y:y+h,x:x+h])

C:\Users\adefo\AppData\Local\Temp/ipykernel_15224/3981687163.py:7: UserWarning: Close the images windows each time one opens 
  warnings.warn("Close the images windows each time one opens ")


## Compute the embedding vectors with pre-trained CNN

In [5]:
#Pre-trained-model

model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [6]:
#Compute the vectors

Vectors = []

for i in range(nb_photos):

    img = image.load_img(rf'Face_extract\\'+your_name +'\\face_' +str(i+1)+ '.png', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=1) # or version=2
    Vectors.append(model.predict(x))

# Step 2 : Detect if your face is in a picture

## Detect all the faces 

In [7]:
# Define path to picture to test

path_to_picture = 'Images\\group_with_adrien'

#Try this path if you want a picture with many people but non from our group

#path_to_picture = 'Images\\im1'

In [8]:
img = cv2.imread(rf''+path_to_picture+'.png')
cv2.imshow('Visualisation',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 10)

# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
faces_identified =[]
for i in range(len(faces)) :
    
    # Save the faces images
    (x, y, w, h) = faces[i]  
    cv2.imwrite(r'Faces_identified\\face_' +str(i)+ '.png', img[y:y+h,x:x+h])    

In [9]:
# Embed them in vectors

for i in range(len(faces)) :
    
    img = image.load_img(rf'Faces_identified\\face_' +str(i)+ '.png', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=1)
    
    faces_identified.append(model.predict(x))

In [10]:
# Function for Checking if face in picture

def is_in_picture(faces_identified, Vectors):
    for element in faces_identified :
        avg =0
        for i in range(nb_photos) :
            avg += np.linalg.norm(np.transpose(element) - np.transpose(Vectors[i]))
        if (avg/nb_photos)<100 :
             return(True)
    return False
            
        

In [11]:
is_in_picture(faces_identified, Vectors)

True